<a href="https://colab.research.google.com/github/abhi1021/resnet50-imagenet-1k/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training ImageNette Model (Google Colab)
This notebook trains the ResNet-50 model on ImageNette-160 (10 classes, epochs=3)

## Download ImageNette Dataset

In [1]:
# Clone the repository
!git clone https://github.com/abhi1021/resnet50-imagenet-1k.git

Cloning into 'resnet50-imagenet-1k'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 120 (delta 56), reused 94 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (120/120), 392.35 KiB | 2.16 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
# Navigate to the repository directory
%cd resnet50-imagenet-1k

/content/resnet50-imagenet-1k


In [3]:
# Install required dependencies
!pip install -q -r requirements.txt
print("✓ Dependencies installed successfully")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✓ Dependencies installed successfully


In [4]:
# Download and extract ImageNette dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
!tar -xzf imagenette2-160.tgz
!ls -la imagenette2-160

--2025-10-18 16:19:43--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.72.112, 52.217.132.192, 52.217.41.78, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.72.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99003388 (94M) [application/x-tar]
Saving to: ‘imagenette2-160.tgz’

imagenette2-160.tgz 100%[===================>]  94.42M  46.9MB/s    in 2.0s    

2025-10-18 16:19:45 (46.9 MB/s) - ‘imagenette2-160.tgz’ saved [99003388/99003388]

total 1228
drwxrwxr-x  4  501 staff    4096 Feb  6  2021 .
drwxr-xr-x  6 root root     4096 Oct 18 16:19 ..
-rw-r--r--  1  501 staff    6148 Feb  6  2021 .DS_Store
-rw-r--r--  1  501 staff 1230799 Feb  6  2021 noisy_imagenette.csv
drwxr-xr-x 12  501 staff    4096 Oct 18 16:19 train
drwxr-xr-x 12  501 staff    4096 Nov 30  2019 val


## Verify Dataset Structure

In [5]:
# Check dataset structure (should have train and val folders)
!ls -la imagenette2-160/train | head -15
!echo "\n"
!ls -la imagenette2-160/val | head -15

total 388
drwxr-xr-x 12 501 staff  4096 Oct 18 16:19 .
drwxrwxr-x  4 501 staff  4096 Feb  6  2021 ..
-rw-r--r--  1 501 staff  6148 Feb  5  2021 .DS_Store
drwxr-xr-x  2 501 staff 40960 Oct 18 16:19 n01440764
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n02102040
drwxr-xr-x  2 501 staff 45056 Nov 30  2019 n02979186
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03000684
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03028079
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03394916
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03417042
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03425413
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03445777
drwxr-xr-x  2 501 staff 36864 Oct 18 16:19 n03888257
\n
total 208
drwxr-xr-x 12 501 staff  4096 Nov 30  2019 .
drwxrwxr-x  4 501 staff  4096 Feb  6  2021 ..
drwxr-xr-x  2 501 staff 20480 Oct 18 16:19 n01440764
drwxr-xr-x  2 501 staff 20480 Oct 18 16:19 n02102040
drwxr-xr-x  2 501 staff 20480 Nov 30  2019 n02979186
drwxr-xr-x  2 501 staff 20480 Oct 18 16:19 n03000684


## Import Libraries and Check GPU

In [6]:
import torch
import sys
from train import ImageClassificationTrainer

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"Memory Cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")
else:
    print("WARNING: Running on CPU. Training will be slower.")
    print("In Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU")

Using device: cuda
GPU: Tesla T4
Memory Allocated: 0.00 MB
Memory Cached: 0.00 MB


## Initialize and Train ImageNette Model with ResNet-50

In [7]:
# Create trainer with specified settings
# Configuration:
#   - Model: ResNet-50 (25.6M parameters)
#   - Dataset: ImageNette (10 classes, 160x160 images)
#   - Epochs: 3
#   - Batch Size: 128 (adjusted for Colab GPU)
#   - Learning Rate: Warmup (0.01→0.1) + Cosine annealing
#   - MixUp augmentation: Enabled (alpha=0.2)
#   - Label smoothing: 0.1
#   - Mixed precision: Enabled
#   - Gradient clipping: 1.0
trainer = ImageClassificationTrainer(
    model_name='resnet50',
    dataset='imagenet',
    data_dir='imagenette2-160',
    epochs=3,
    batch_size=128,
    num_workers=2
)

print("Starting ImageNette Model Training with ResNet-50")
print("="*70)
print(f"Configuration:")
print(f"  - Model: ResNet-50 (25.6M parameters)")
print(f"  - Dataset: ImageNette (10 classes)")
print(f"  - Image Size: 160x160 (resized to 224x224)")
print(f"  - Batch Size: 128")
print(f"  - Epochs: 3")
print(f"  - Optimizer: SGD (momentum=0.9, weight_decay=1e-3)")
print(f"  - Learning Rate: Warmup (0.01→0.1) + Cosine annealing")
print(f"  - MixUp: Enabled (alpha=0.2)")
print(f"  - Label Smoothing: 0.1")
print(f"  - Mixed Precision: Enabled")
print(f"  - Gradient Clipping: 1.0")
print("="*70)

✓ Using NVIDIA GPU: Tesla T4

Loading ImageNet dataset from: imagenette2-160
ImageNet dataset loaded:
  Training samples: 9,469
  Validation samples: 3,925
  Number of classes: 10
Detected 10 classes in dataset
Starting ImageNette Model Training with ResNet-50
Configuration:
  - Model: ResNet-50 (25.6M parameters)
  - Dataset: ImageNette (10 classes)
  - Image Size: 160x160 (resized to 224x224)
  - Batch Size: 128
  - Epochs: 3
  - Optimizer: SGD (momentum=0.9, weight_decay=1e-3)
  - Learning Rate: Warmup (0.01→0.1) + Cosine annealing
  - MixUp: Enabled (alpha=0.2)
  - Label Smoothing: 0.1
  - Mixed Precision: Enabled
  - Gradient Clipping: 1.0


In [8]:
# Run training
final_accuracy = trainer.run()


Training resnet50 for 3 epochs on ImageNet (10 classes)

Model Architecture Summary
Device: cuda
Model: resnet50
Number of classes: 10
Mixed Precision: True
MixUp: True (alpha=0.2)
Label Smoothing: 0.1

Total Parameters: 23,528,522
Trainable Parameters: 23,528,522
Non-trainable Parameters: 0

Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]  

Epoch 1 Loss=2.3656 Acc=11.27% LR=0.027757: 100%|██████████| 74/74 [00:50<00:00,  1.46it/s]



Test set: Average loss: 2.4451, Accuracy: 417/3925 (10.62%)

*** New best model! Test Accuracy: 10.62% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 10.62%



Epoch 2 Loss=2.4061 Acc=11.96% LR=0.045757: 100%|██████████| 74/74 [00:51<00:00,  1.43it/s]



Test set: Average loss: 2.4954, Accuracy: 636/3925 (16.20%)

*** New best model! Test Accuracy: 16.20% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 16.20%



Epoch 3 Loss=2.5788 Acc=16.76% LR=0.063757: 100%|██████████| 74/74 [00:50<00:00,  1.46it/s]



Test set: Average loss: 2.3827, Accuracy: 687/3925 (17.50%)

*** New best model! Test Accuracy: 17.50% ***
✓ Checkpoint saved: best_model.pth
Best Test Accuracy so far: 17.50%


📦 Saving final model...
✓ Checkpoint saved: final_model.pth

Training completed. Best test accuracy: 17.50%

TRAINING AND TESTING LOSS
   (Y)     ^
2.74497202 |
2.60772342 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.47047482 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⡠⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⠤⢄⣀⣀⣀⣀⣀⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.33322622 | ⣉⣉⣉⣉⣉⣉⣉⣉⣉⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣈⣉⣉⣉⣉⠭⠭⠭⠭⠶⠶⠶⠶⠶⠶⠶⠶⠶⠤⠤⠤⠤
2.19597762 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2.05872901 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.92148041 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.78423181 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
1.64698321 | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Training Summary

In [9]:
print("\n" + "="*70)
print("TRAINING COMPLETED")
print("="*70)
print(f"Model: ResNet-50")
print(f"Dataset: ImageNette (10 classes)")
print(f"Final Best Test Accuracy: {final_accuracy:.2f}%")
print("="*70)
print("\nCheckpoints saved in ./checkpoints/")
print("  - best_model.pth (highest test accuracy)")
print("  - final_model.pth (final epoch)")
print("  - training_curves.png (visualization)")
print("  - metrics.json (complete training history)")


TRAINING COMPLETED
Model: ResNet-50
Dataset: ImageNette (10 classes)
Final Best Test Accuracy: 17.50%

Checkpoints saved in ./checkpoints/
  - best_model.pth (highest test accuracy)
  - final_model.pth (final epoch)
  - training_curves.png (visualization)
  - metrics.json (complete training history)
